# Financial Report Analysis using LLM
## Problem Statement: Financial Image Scans

This notebook demonstrates an AI-powered solution for analyzing financial documents by:
- Scanning and extracting insights from financial document images
- Analyzing charts, graphs, and tables
- Generating comprehensive summaries with key metrics, trends, and risks
- Providing actionable insights for investors, analysts, and decision-makers

## Step 1: Install Required Libraries

In [ ]:
# Install required packages for OpenAI and image processing
!pip install openai pillow requests --quiet

## Step 2: Import Libraries

In [ ]:
import openai
import base64
import os
from PIL import Image
import io
import requests
from pathlib import Path

## Step 3: Configure API Key

You'll need to set your OPENAI API key.

In [ ]:
# Set your OpenAI API key here
OPENAI_API_KEY = "your-openai-api-key-here"  # Replace with your actual OpenAI API key

# Or use environment variable
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", OPENAI_API_KEY)

openai.api_key = OPENAI_API_KEY

## Step 4: Helper Functions for Image Processing

In [ ]:
def load_image_from_path(image_path):
    """
    Load an image from local file path and convert to base64
    """
    with open(image_path, "rb") as image_file:
        image_data = base64.standard_b64encode(image_file.read()).decode("utf-8")
    
    # Determine media type based on file extension
    extension = Path(image_path).suffix.lower()
    media_type_map = {
        '.jpg': 'image/jpeg',
        '.jpeg': 'image/jpeg',
        '.png': 'image/png',
        '.gif': 'image/gif',
        '.webp': 'image/webp'
    }
    media_type = media_type_map.get(extension, 'image/jpeg')
    
    return image_data, media_type


def load_image_from_url(image_url):
    """
    Load an image from URL and convert to base64
    """
    response = requests.get(image_url)
    image_data = base64.standard_b64encode(response.content).decode("utf-8")
    
    # Determine media type from response headers or URL
    content_type = response.headers.get('content-type', 'image/jpeg')
    
    return image_data, content_type


def load_multiple_images(image_sources):
    """
    Load multiple images from paths or URLs
    
    Args:
        image_sources: List of file paths or URLs
    
    Returns:
        List of tuples (image_data, media_type)
    """
    images = []
    
    for source in image_sources:
        if source.startswith('http://') or source.startswith('https://'):
            image_data, media_type = load_image_from_url(source)
        else:
            image_data, media_type = load_image_from_path(source)
        
        images.append((image_data, media_type))
        print(f"✓ Loaded image: {source}")
    
    return images

## Step 5: Financial Report Analysis Function

In [ ]:
def analyze_financial_report(images, custom_prompt=None):
    """
    Analyze financial report images using OpenAI Vision API
    
    Args:
        images: List of tuples (image_data, media_type)
        custom_prompt: Optional custom prompt for specific analysis needs
    
    Returns:
        Analysis summary from OpenAI Vision model
    """
    
    # Default comprehensive prompt for financial analysis
    default_prompt = """Please analyze these financial document images and provide a comprehensive summary.

Your analysis should include:

1. **Key Financial Metrics:**
   - Revenue, Net Income, EPS (Earnings Per Share)
   - Growth trends compared to previous periods
   - Profitability, liquidity, and leverage ratios

2. **Financial Statements Summary:**
   - Balance Sheet: Assets, Liabilities, Equity
   - Income Statement: Revenue, Expenses, Net Income
   - Cash Flow Statement: Operating, Investing, Financing activities

3. **Operational Performance:**
   - Key operational metrics and KPIs
   - Market conditions and competitive position
   - Management commentary and strategic priorities

4. **Charts and Graphs Analysis:**
   - Interpret all visual data representations
   - Identify trends, patterns, and anomalies
   - Highlight significant changes or inflection points

5. **Risks and Challenges:**
   - Identified risks and mitigation strategies
   - External factors affecting business
   - Regulatory or compliance issues

6. **Notable Events:**
   - Acquisitions, partnerships, or major initiatives
   - Changes in leadership or strategy
   - Shareholder information (dividends, stock performance)

7. **Actionable Insights:**
   - Key takeaways for investors and decision-makers
   - Areas requiring attention or monitoring
   - Strategic recommendations based on the data

Please provide a clear, concise, and well-structured analysis."""
    
    prompt = custom_prompt if custom_prompt else default_prompt
    
    # Prepare the content array with text and images for OpenAI API
    content = []
    
    # Add all images first (OpenAI expects base64-encoded images as data URLs)
    for idx, (image_data, media_type) in enumerate(images):
        data_url = f"data:{media_type};base64,{image_data}"
        content.append({
            "type": "image_url",
            "image_url": {"url": data_url}
        })
    
    # Add the text prompt
    content.append({
        "type": "text",
        "text": prompt
    })
    
    print("\n🔄 Analyzing financial documents with OpenAI Vision API...\n")
    
    response = openai.chat.completions.create(
        model="gpt-4-vision-preview",
        max_tokens=4096,
        messages=[
            {
                "role": "user",
                "content": content
            }
        ]
    )
    
    return response.choices[0].message.content

## Step 6: Example Usage - Load Financial Report Images

You can load images from:
1. Local file paths
2. URLs (publicly accessible images)

**Note:** Replace the example sources below with your actual financial document images.

In [ ]:
# Example 1: Using sample financial report images (URLs)
# Replace these with your actual financial document images

image_sources = [
    # Example: Annual report cover page
    "https://example.com/financial-report-page1.png",
    
    # Example: Balance sheet
    "https://example.com/balance-sheet.png",
    
    # Example: Income statement with charts
    "https://example.com/income-statement.png",
]

# OR use local file paths:
# image_sources = [
#     "/path/to/financial_report_page1.png",
#     "/path/to/balance_sheet.png",
#     "/path/to/income_statement.png",
# ]

# Load the images
# images = load_multiple_images(image_sources)

## Step 7: Demo with Sample Financial Report

For demonstration purposes, let's use publicly available financial report images.
In practice, you would replace these with your actual financial documents.

In [ ]:
# Demo: Create sample images or use public financial reports
# For this demo, we'll create placeholder instructions

print("="*80)
print("DEMO: Financial Report Analysis")
print("="*80)
print("\nTo run this analysis:")
print("1. Replace 'image_sources' with paths to your financial document images")
print("2. Ensure images include: balance sheets, income statements, charts, etc.")
print("3. Uncomment the code in the cells below")
print("4. Run the analysis\n")

# Uncomment below when you have actual images:
# images = load_multiple_images(image_sources)
# analysis_result = analyze_financial_report(images)
# print("\n" + "="*80)
# print("ANALYSIS RESULTS")
# print("="*80 + "\n")
# print(analysis_result)

## Step 8: Example Output Format

Below is an example of what the LLM output should look like when analyzing financial reports:

In [ ]:
example_output = """
EXAMPLE OUTPUT:
================================================================================

Summary of XYZ Corporation Quarterly Report (Q3 2024)

Key Financial Metrics
• Net Profit: $850 million, up 12% from Q3 2023 ($758 million)
• Earnings Per Share (EPS): $2.45, increased from $2.18 in Q3 2023
• Return on Equity: 15.3%, exceeding the target range of 12-14%
• Operating Margin: 18.5%, improved from 16.8% year-over-year
• Revenue: $5.2 billion, representing 8% YoY growth

Income and Expenses
• Total Revenue: Increased 8% YoY to $5.2 billion, driven by strong product sales
• Gross Profit: $2.1 billion with gross margin of 40.4%
• Operating Expenses: $1.14 billion, up 5% due to R&D investments
• Net Income: $850 million, showing strong profitability

Balance Sheet Highlights
• Total Assets: $42.3 billion, increase of $3.2 billion from Q2 2024
• Cash and Equivalents: $8.5 billion, providing strong liquidity
• Total Liabilities: $18.7 billion
• Shareholders' Equity: $23.6 billion
• Debt-to-Equity Ratio: 0.45, indicating healthy leverage

Cash Flow Analysis
• Operating Cash Flow: $1.2 billion, up 15% YoY
• Free Cash Flow: $890 million after capital expenditures
• Capital Expenditures: $310 million invested in growth initiatives

Charts and Graphs Insights
• Revenue trend chart shows consistent upward trajectory over past 5 quarters
• Regional performance graph indicates 25% growth in Asia-Pacific markets
• Product mix chart reveals diversification with 40% from new product lines
• Customer acquisition graph shows 18% increase in active users

Strategic and Operational Updates
• Launched 3 new products contributing $180 million in revenue
• Completed acquisition of Tech Innovations Inc. for $450 million
• Expanded operations to 5 new markets in Southeast Asia
• Increased R&D spending by 12% to drive innovation

Risks and Challenges
• Supply chain disruptions in certain regions
• Increased competition in core markets
• Regulatory changes in international markets
• Currency fluctuation impact on international revenue
• Mitigation: Diversifying supplier base and hedging strategies

Market Conditions and Outlook
• Market share increased to 23%, up from 21% in previous year
• Industry growth rate projected at 6-8% annually
• Management forecasts 10-12% revenue growth for full year 2024
• Digital transformation initiatives expected to drive efficiency

Shareholder Information
• Quarterly dividend: $0.85 per share, 6% increase from previous year
• Stock price appreciation: 22% year-to-date
• Share buyback program: $500 million authorized

Actionable Insights for Investors
1. Strong financial performance with consistent growth trajectory
2. Healthy balance sheet with low leverage provides financial flexibility
3. Successful product innovation and market expansion strategies
4. Monitor supply chain risks and competitive pressures
5. Consider long-term hold position given strong fundamentals
6. Watch Q4 results for confirmation of full-year guidance

Financial Ratios Summary
• Current Ratio: 2.1 (healthy liquidity)
• Quick Ratio: 1.6 (strong short-term solvency)
• ROA: 8.2% (efficient asset utilization)
• ROE: 15.3% (strong returns to shareholders)
• Debt-to-Equity: 0.45 (conservative leverage)

This comprehensive analysis demonstrates XYZ Corporation's robust Q3 2024 
performance, characterized by strong revenue growth, improved profitability, 
strategic expansion initiatives, and solid financial position despite external 
challenges. The company appears well-positioned for continued growth.
"""

print(example_output)

## Step 9: Custom Analysis for Specific Use Cases

You can customize the analysis prompt for different stakeholder needs:

In [ ]:
# Example custom prompts for different users

investor_prompt = """
As an investor analyzing this financial report, please focus on:
1. Investment potential and risk assessment
2. Dividend policy and shareholder returns
3. Growth trajectory and future outlook
4. Competitive advantages and market position
5. Management effectiveness and strategic direction
6. Key ratios: P/E, ROE, ROA, debt-to-equity
7. Any red flags or concerns for investors
"""

auditor_prompt = """
As an auditor reviewing this financial report, please analyze:
1. Compliance with accounting standards
2. Accuracy and consistency of financial statements
3. Internal controls and governance indicators
4. Revenue recognition practices
5. Asset valuations and depreciation methods
6. Debt obligations and contingent liabilities
7. Any discrepancies or areas requiring further investigation
"""

analyst_prompt = """
As a financial analyst, provide a detailed analysis covering:
1. Comprehensive ratio analysis (profitability, liquidity, efficiency)
2. Trend analysis across multiple periods
3. Peer comparison and industry benchmarking
4. Valuation metrics and stock performance
5. Risk factors and sensitivity analysis
6. Forward-looking projections based on current data
7. Investment recommendation with supporting rationale
"""

# Uncomment to use custom prompts:
# analysis_result = analyze_financial_report(images, custom_prompt=investor_prompt)
# print(analysis_result)

## Step 10: Save Analysis Results

In [ ]:
def save_analysis_to_file(analysis_text, filename="financial_analysis_report.txt"):
    """
    Save the analysis results to a text file
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(analysis_text)
    print(f"✓ Analysis saved to {filename}")

# Uncomment to save results:
# save_analysis_to_file(analysis_result, "my_financial_analysis.txt")

## Conclusion

This notebook demonstrates a complete solution for AI-powered financial report analysis:

**Key Features:**
- ✅ Loads images from local paths or URLs
- ✅ Analyzes financial statements, charts, and graphs
- ✅ Extracts key metrics, trends, and risks
- ✅ Provides actionable insights for different stakeholders
- ✅ Customizable prompts for specific analysis needs
- ✅ Comprehensive output covering all financial aspects

**To use this solution:**
1. Set your OPENAI API key
2. Provide paths or URLs to your financial document images
3. Run the analysis function
4. Review the comprehensive AI-generated insights

**Benefits:**
- Saves hours of manual document review
- Reduces risk of overlooking critical information
- Provides consistent, structured analysis
- Enables faster, more informed decision-making
- Accessible to users without deep financial expertise